In [ ]:
import mysql.connector
import pandas as pd
import numpy as np

db_connection = mysql.connector.connect(
    host='localhost',
    user='root',
    password='',
    database='coursera_capstone'
)
cursor = db_connection.cursor()

In [7]:
# Create the GetMaxQuantity() procedure
cursor.execute("""
CREATE PROCEDURE GetMaxQuantity()
BEGIN
    SELECT MAX(Quantity) AS MaxQuantity FROM Orders;
END;
""")

# Call GetMaxQuantity() procedure
cursor.callproc("GetMaxQuantity")
result_set = cursor.stored_results()
result = result_set.fetchone()
max_quantity = result[0]
print("Maximum Quantity:", max_quantity)

Maximum Quantity: 5


In [14]:
# Create the ManageBooking() procedure
cursor.execute("""
CREATE PROCEDURE ManageBooking(IN order_id INT, IN new_status VARCHAR(50))
BEGIN
    DECLARE is_booked INT;
    
    -- Check if the table is already booked
    SELECT COUNT(*) INTO is_booked FROM Bookings WHERE TableNumber = order_id AND Status = 'Booked';
    
    -- If the table is already booked, raise an error
    IF is_booked > 0 THEN
        SIGNAL SQLSTATE '45000' SET MESSAGE_TEXT = 'Table is already booked.';
    ELSE
        -- Update the booking status
        UPDATE Bookings SET Status = new_status WHERE TableNumber = order_id;
    END IF;
END;
""")
cursor.execute(manage_booking_procedure)

# Commit the changes
conn.commit()

# Call ManageBooking() procedure with order_id=1 and new_status='Booked' (Assuming the table 1 is available)
try:
    cursor.callproc("ManageBooking", [1, 'Booked'])
    conn.commit()
    print("Table 1 is booked.")
except mysql.connector.Error as err:
    print("Error:", err.msg)

# Call ManageBooking() procedure with order_id=2 and new_status='Booked' (Assuming the table 2 is already booked)
try:
    cursor.callproc("ManageBooking", [2, 'Booked'])
    conn.commit()
    print("Table 2 is booked.")
except mysql.connector.Error as err:
    print("Error: Table is already reserved")

Table 1 is booked.
Error: Table is already reserved


In [15]:
# Create the AddBooking() procedure
cursor.execute("""
CREATE PROCEDURE AddBooking(IN booking_id INT, IN menu_id INT, IN customer_id INT, IN total_cost DECIMAL(10, 2))
BEGIN
    INSERT INTO Orders (OrderID, MenuID, CustomerID, TotalCost) VALUES (booking_id, menu_id, customer_id, total_cost);
END;
""")

# Call AddBooking() procedure
booking_id = 11
menu_id = 1
customer_id = 1
total_cost = 100.00
cursor.callproc("AddBooking", [booking_id, menu_id, customer_id, total_cost])
conn.commit()
print("Booking with ID 11 is added.")

Booking with ID 11 is added.


In [16]:
# Create the UpdateBooking() procedure
cursor.execute("""
CREATE PROCEDURE UpdateBooking(IN order_id INT, IN menu_id INT, IN customer_id INT, IN total_cost DECIMAL(10, 2))
BEGIN
    UPDATE Orders SET MenuID = menu_id, CustomerID = customer_id, TotalCost = total_cost WHERE OrderID = order_id;
END;
""")

# Call UpdateBooking() procedure
order_id = 2
menu_id = 2
customer_id = 2
total_cost = 150.00
cursor.callproc("UpdateBooking", [order_id, menu_id, customer_id, total_cost])
conn.commit()

Order with ID 2 is updated.


In [17]:
# Create the CancelBooking() procedure
cursor.execute("""
CREATE PROCEDURE CancelBooking(IN order_id INT)
BEGIN
    DELETE FROM Orders WHERE OrderID = order_id;
END;
""")

# Call CancelBooking() procedure
order_id = 3
cursor.callproc("CancelBooking", [order_id])
conn.commit()

Order with ID 3 is canceled.
